线性回归的简洁实现

In [5]:
import torch
import d2l
import numpy as np
from torch.utils import data

生成数据集

In [6]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用API读取数据集

In [7]:
def load_array(data_arrays, batch_size, is_train=True):     #@save
    """ 构造一个基于Pytorch的数据迭代器 """
    dataset = data.TensorDataset(*data_arrays)  # 表明datat_attays是一个元组
    return data.DataLoader(dataset, batch_size, shuffle=is_train)   # is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据

batch_size = 10
data_iter = load_array((features, labels), batch_size)

验证数据迭代器是否正常工作

In [8]:
next(iter(data_iter))

[tensor([[ 0.2348, -0.6778],
         [ 1.0357, -0.1975],
         [-1.1187, -1.4215],
         [ 0.0192, -1.5897],
         [ 0.1269,  0.0876],
         [ 2.5652, -0.4655],
         [-0.6237,  0.0599],
         [ 0.0806, -1.0334],
         [ 0.5170,  2.3261],
         [ 0.7248, -1.0568]]),
 tensor([[ 6.9658],
         [ 6.9335],
         [ 6.7877],
         [ 9.6526],
         [ 4.1684],
         [10.9116],
         [ 2.7545],
         [ 7.8775],
         [-2.6872],
         [ 9.2379]])]

定义模型，首先定义一个模型变量net，它是一个Sequential类的实例，Sequential类为串联在一起的多个层定义了一个容器。当给定输入数据，Sequential类实例将数据传入第一层，然后第一层的输出作为第二层的输入，以此类推。

我们目前的线性回归模型是只有一层的网络架构，这一单层被称为**全连接层**，因为它的每一个输入都是通过矩阵-向量乘法连接到它的每个输出。

In [9]:
# Pytorch中，全连接层在Linear类中定义
# 将两个参数传递到nn.Linear中，第一个为输入的特征形状，第二个为输出的特征形状

from torch import nn    # nn是神经网络neural network的缩写

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数。

net[0]表示选择网络中的第一个图层，即我们的线性全连接层，然后定义权重weight和偏置bias的值。

In [10]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

定义损失函数。

使用的是MSELoss类，默认情况下会返回所有样本损失的均值。

In [11]:
loss = nn.MSELoss()

定义优化算法。

Pytorch在optim模块实现了该算法的许多变种。

In [12]:
# 需要优化的参数通过net.parameters()中获得
# 小批量随机梯度下降只需要设置lr的值
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练。

In [14]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1},loss {l:f}')

epoch 1,loss 0.000098
epoch 2,loss 0.000097
epoch 3,loss 0.000097


In [16]:
w = net[0].weight.data
print('w的估计误差: ', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差: ', true_b - b)

w的估计误差:  tensor([-0.0003, -0.0007])
b的估计误差:  tensor([-0.0002])
